In [16]:
!pip install transformers --quiet
!pip install fasttext --quiet
!pip install simpletransformers --quiet

     |████████████████████████████████| 204kB 5.2MB/s 
     |████████████████████████████████| 7.4MB 13.5MB/s 
     |████████████████████████████████| 1.8MB 48.9MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 1.1MB 41.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 317kB 55.6MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 112kB 62.8MB/s 
     |████████████████████████████████| 163kB 52.6MB/s 
     |████████████████████████████████| 4.5MB 57.2MB/s 
     |████████████████████████████████| 133kB 58.8MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 122kB 52.9MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.2 which is incompatible.


In [17]:
from nltk.corpus import brown
import nltk

import string
import os, errno

import fasttext
import csv

import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torchtext.data import Field, TabularDataset, BucketIterator
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

from simpletransformers.classification import ClassificationModel

In [3]:
# Download the Brown corpus
nltk.download('brown')

# Assert that necessary Python files are in current directory
assert os.path.exists('./createModel.py')
assert os.path.exists('./demoBERT.py')
assert os.path.exists('./demoRoBERTa.py')

# Create a 'lstm' folder if it doesn't exist
try:
  os.makedirs('lstm')
except OSError as e:
  if e.errno != errno.EEXIST:
    raise

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [4]:
#from createModel import *
#from demoBERT import *
from demoRoBERTa import *
from classifier_helpers import *

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ranks before : [1783, 15334, 1255, 6336, 9214, 10334, 3409, 261, 1622, 52]
cover_text:  Hello world, today we are going to present you a tutorial about how to cook a turkey. Please follow this tutorial carefully,  suspKKpe replied threatens shootingnick […] Example and

ranks after: [1783, 15334, 1255, 6336, 9214, 10334, 3409, 261, 1622, 52]
. this message is super secret please do not share it


In [5]:
mod, tok = buildModelRoBERTa()

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
start_text = """
Welcome to the Shakespeare festival in London, UK. Today
"""

start_secret = "Secret"
secret_text = "A secret message for Joe"

In [7]:
print(gen_fake_text(mod, tok, start_text, start_secret, secret_text))


Welcome to the Shakespeare festival in London, UK. Today
Youadj Securities Predictrael


In [8]:
NUM_OF_ARTICLES = 100
true_news, fake_news = get_news(mod, tok, 
                                start_secret, 
                                secret_text, 
                                corpus_size=NUM_OF_ARTICLES, 
                                secret_text_length=5)

1 / 100
2 / 100
3 / 100
4 / 100
5 / 100
6 / 100
7 / 100
8 / 100
9 / 100
10 / 100
11 / 100
12 / 100
13 / 100
14 / 100
15 / 100
16 / 100
17 / 100
18 / 100
19 / 100
20 / 100
21 / 100
22 / 100
23 / 100
24 / 100
25 / 100
26 / 100
27 / 100
28 / 100
29 / 100
30 / 100
31 / 100
32 / 100
33 / 100
34 / 100
35 / 100
36 / 100
37 / 100
38 / 100
39 / 100
40 / 100
41 / 100
42 / 100
43 / 100
44 / 100
45 / 100
46 / 100
47 / 100
48 / 100
49 / 100
50 / 100
51 / 100
52 / 100
53 / 100
54 / 100
55 / 100
56 / 100
57 / 100
58 / 100
59 / 100
60 / 100
61 / 100
62 / 100
63 / 100
64 / 100
65 / 100
66 / 100
67 / 100
68 / 100
69 / 100
70 / 100
71 / 100
72 / 100
73 / 100
74 / 100
75 / 100
76 / 100
77 / 100
78 / 100
79 / 100
80 / 100
81 / 100
82 / 100
83 / 100
84 / 100
85 / 100
86 / 100
87 / 100
88 / 100
89 / 100
90 / 100
91 / 100
92 / 100


In [9]:
true_news, fake_news = prepare_news(true_news, fake_news)

the jury further said in termend presentments that the city executive committee  which had overall charge of the election   deserves the praise and thanks of the city of atlanta  for the manner in which the election was conducted 


In [10]:
true_news

['the fulton county grand jury said friday an investigation of atlantas recent primary election produced  no evidence  that any irregularities took place',
 'the jury further said in termend presentments that the city executive committee  which had overall charge of the election   deserves the praise and thanks of the city of atlanta  for the manner in which the election was conducted',
 'the septemberoctober term jury had been charged by fulton superior court judge durwood pye to investigate reports of possible  irregularities  in the hardfought primary which was won by mayornominate ivan allen jr',
 'only a relative handful of such reports was received   the jury said   considering the widespread interest in the election  the number of voters and the size of this city',
 'the jury said it did find that many of georgias registration and election laws  are outmoded or inadequate and often ambiguous',
 'it recommended that fulton legislators act  to have these laws studied and revised t

In [11]:
destination_folder = './lstm/'

train_test_ratio = 0.90
train_valid_ratio = 0.80

first_n_words = 200

def trim_string(x):
    x = x.split(maxsplit=first_n_words)
    x = ' '.join(x[:first_n_words])
    return x

In [12]:
df = prepare_df(true_news, fake_news)
df

,label,text
0,0,the fulton county grand jury said friday an in...
1,0,the jury further said in termend presentments ...
2,0,the septemberoctober term jury had been charge...
3,0,only a relative handful of such reports was re...
4,0,the jury said it did find that many of georgia...
...,...,...
179,1,many local citizens feared that there would be...
180,1,sheriff felix tabb said the ordinary apparentl...
181,1,everything went real smooth security banner li...
182,1,there wasnt a signing bustling bast winterpatrick


# fasttext

In [13]:
prepare_df_for_fasttext(df, train_test_ratio)

In [14]:
model = fasttext.train_supervised('train.txt', wordNgrams = 200)

In [15]:
model.test('test.txt')

(19, 0.42105263157894735, 0.42105263157894735)

# Transformer Classifier

pandas.core.series.Series